In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from imutils import paths
import numpy as np
import pandas as pd
import argparse
import imutils
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
def image_to_feature_vector(image, size=(64,64)):
    return cv2.resize(image,size).flatten()

def extract_color_histogram(image, bins=(8,8,8)):
    # extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0,1,2], None, bins, [0,180,0,256,0,256])
    
	# handle normalizing the histogram if we are using OpenCV 2.4.X
    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    # otherwise, perform "in place" normalization in OpenCV 3 (I
	# personally hate the way this is done
    else:
        cv2.normalize(hist,hist)
    # return the flattened histogram as the feature vector
    return hist.flatten()

In [3]:
# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
rawImages = []
features = []
labels = []

# grab the list of images that we'll be describing
print("[INFO] describing images...")
os.chdir(os.getcwd()+'\\flowers')
print(os.getcwd())

[INFO] describing images...
D:\Desktop Folder\Self Projects\Flower Recognition\flowers


In [4]:
i = 0
for x in os.walk(os.getcwd()):
    if i == 0:
        i += 1
        continue
    flower_label = x[0].split('\\')[-1]
    for flowers in x[1:]:
        for flower in flowers:
            imagePath = os.getcwd()+f"\\{flower_label}"+f"\\{flower}"
            image = cv2.imread(imagePath)
            pixels = image_to_feature_vector(image)
            hist = extract_color_histogram(image)
            
            rawImages.append(pixels)
            features.append(hist)
            labels.append(flower_label)
    print(f"{flower_label} Processed")

daisy Processed
dandelion Processed
rose Processed
sunflower Processed
tulip Processed


In [5]:
(trainRI, testRI, trainRL, testRL) = train_test_split(rawImages, labels, test_size =0.25, stratify=labels)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(features, labels, test_size =0.25, stratify=labels)

print("[INFO] Evaluating Raw Pixel Accuracy")
model = KNeighborsClassifier(n_neighbors=4, n_jobs=-1)
model.fit(trainFeat,trainLabels)
acc = model.score(testFeat,testLabels)

print(f"[INFO] histogram accuracy: {acc*100:.2f}")

[INFO] Evaluating Raw Pixel Accuracy
[INFO] histogram accuracy: 45.19


In [6]:
cf_matrix = multilabel_confusion_matrix(model.predict(testFeat), testLabels, labels=['daisy', 'dandelion','rose', 'sunflower', 'tulip'])

print(cf_matrix)

[[[718  92]
  [171  99]]

 [[663 146]
  [154 117]]

 [[788 117]
  [ 96  79]]

 [[828  87]
  [ 68  97]]

 [[731 150]
  [103  96]]]


In [7]:
import sklearn
report = sklearn.metrics.classification_report(testLabels, model.predict(testFeat), output_dict=True)
df = pd.DataFrame(report).transpose()
print(df)

              precision    recall  f1-score      support
daisy          0.366667  0.518325  0.429501   191.000000
dandelion      0.431734  0.444867  0.438202   263.000000
rose           0.451429  0.403061  0.425876   196.000000
sunflower      0.587879  0.527174  0.555874   184.000000
tulip          0.482412  0.390244  0.431461   246.000000
accuracy       0.451852  0.451852  0.451852     0.451852
macro avg      0.464024  0.456734  0.456183  1080.000000
weighted avg   0.461947  0.451852  0.452939  1080.000000
